### Portail meteo.data.gouv.fr - Carte des postes météorologiques de Météo-France (Métropole & outre-mer)
Une connexion internet est nécessaire pour accéder au fichier JSON et au fond de carte<br>
NB: les fichier JSON est incomplet à ce jour (les données comportent des postes absents de la liste JSON)

- La carte des poste est tracée à partir du fichier JSON de Météo-France qui contient les noms du poste et de la commune, ainsi que l'alitude<br>
- La carte dynamique est enregistrée en Html, ainsi qu'en version png
- La liste est enregistrée au format excel
- NB: code adapté au format du fichier JSON en décembre 2023

data : https://meteo.data.gouv.fr/ (6 min, horaire, quotidien, mensuel)<br>
Fiche PDF des postes : https://www.data.gouv.fr/fr/datasets/r/bee4b0c7-260a-40fe-b463-ed5631d6dc39 (paramètres et périodes de mesure)

Auteur: https://github.com/loicduffar


##### Lecture du fichier JSON 

In [45]:
############ Auteur: L. Duffar ###########
############ Décembre 2023 ###########
# python 3.8.12
# Description:  Extraction des données du fichier JSON des stations météo de Météo-France
import pandas as pd
import os
import datetime
import json
import requests
import sys
import plotly.express as px

# ================ Personalisation ====================
# Chemin d'enregistrement de la carte et du fichier excel
folder_out= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base"

# Chemin d'accès local au fichier local JSON +++++++++++++ INUTILE TANT QUE LE FICHIER EN LIGNE EST DISPONIBLE
# folder_in= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base"
# Nom du fichier JSON des stations
# file_meta= 'fiches.json'

# ================ Initialisation ====================
# https://donneespubliques.meteofrance.fr/metadonnees_publiques/fiches/fiches.json
url_liste_postes= "https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922"

# ================ Extrait du fichier JSON ==================== Le fichier JSON ne respecte pas le standard GeoJSON (à ce jour du moins)
# {
#     "crs": {
#         "type": "name",
#         "properties": {
#             "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
#         }
#     },
#     "type": "featureCollection",
#     "features": {
#         "0": {
#             "geometry": {
#                 "type": "Point",
#                 "coordinates": [
#                     5.669,
#                     46.278167
#                 ]
#             },
#             "type": "Feature",
#             "properties": {
#                 "COMMUNE": "ARBENT",
#                 "NUM_POSTE": "01014002",
#                 "ALTI": 534,
#                 "NOM_USUEL": "ARBENT",
#                 "LAT_DG": 46.278167,
#                 "NUM_DEP": 1,
#                 "LON_DG": 5.669,
#                 "ficheClimComplete": 0,
#                 "ficheClimReduite": 1
#             }
#         },

# ================ Traitement ==================== 
# Lecture en ligne du fichier JSON
req= requests.get(url_liste_postes)
if req.status_code == 200:
    data_json= req.json()
else:
    print('la requête a échoué avec le code : ', req.status_codes)
    sys.exit() # interrompt le script

# Ouverture du fichier JSON en local
# with open(os.path.join(folder_in, file_meta), encoding='utf-8') as json_file:
#     data = json.load(data)

df = pd.DataFrame(data_json['features']).T
# Ajout de colonnes avec les champs de properties
df['lat'] = df['geometry'].apply(lambda x: x['coordinates'][1]).astype(float)
df['lon'] = df['geometry'].apply(lambda x: x['coordinates'][0]).astype(float)
df['nom_usuel']= df['properties'].apply(lambda x: x['NOM_USUEL'].strip())
df['num_poste']= df['properties'].apply(lambda x: x['NUM_POSTE'].strip())
df['commune'] = df['properties'].apply(lambda x: x['COMMUNE'].strip())
df['ficheClimComplete'] = df['properties'].apply(lambda x: x['ficheClimComplete']).astype(float)
df['ficheClimReduite'] = df['properties'].apply(lambda x: x['ficheClimReduite']).astype(float)
df['alti'] = df['properties'].apply(lambda x: x['ALTI'])

# sauvegarde dans un fichier excel plus lisible que le JSON après suppression des colonnes type, geometry et properties
df.drop(['type', 'geometry', 'properties'], axis=1, inplace=True)
df.sort_values(by= ['nom_usuel'], inplace=True)

# affiche la date et l'heure de l'extraction
now= datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M"))
df

2023-12-25 11:23


,lat,lon,nom_usuel,num_poste,commune,ficheClimComplete,ficheClimReduite,alti
1909,50.136000,1.834000,ABBEVILLE,80001001,ABBEVILLE,1.0,1.0,69
1507,42.945167,-0.610500,ACCOUS,64006001,LESCUN,0.0,0.0,495
1874,48.983167,2.126000,ACHERES,78005002,SAINT-GERMAIN-EN-LAYE,0.0,0.0,31
613,47.169667,6.394667,ADAM-LES-VERCEL,25007001,ADAM-LES-VERCEL,0.0,1.0,665
1541,42.972833,-0.072167,ADAST,65001001,ADAST,0.0,0.0,446
...,...,...,...,...,...,...,...,...
2146,49.108167,1.830667,WY-DIT,95690001,WY-DIT-JOLI-VILLAGE,0.0,0.0,126
1173,44.205000,0.261167,XAINTRAILLES,47327001,XAINTRAILLES,0.0,0.0,190
2389,-22.156500,166.939000,YATE MRIE,98832002,YATE,0.0,1.0,25
1102,45.135167,4.127500,YSSINGEAUX,43268004,YSSINGEAUX,0.0,1.0,873


#### Trace la carte des postes et la sauvegarde en local

In [63]:
# trace une carte plotly des points

fig = px.scatter_mapbox(df, 'lat', lon= 'lon', 
                        hover_name= "nom_usuel", hover_data= ["num_poste", 'alti', "commune", 'ficheClimComplete', 'ficheClimReduite'], 
                        zoom=5, height= 800, width= 900, 
                        )

now= datetime.datetime.now()
title= 'Stations Météo-France (meteo.data.gouv.fr) ' + now.strftime("%Y-%m-%d")
fig.update_layout(title_text= title, title_x=0.5  , hoverlabel= dict(bgcolor= 'rgb(255,255,255)'))
fig.update_traces(marker=dict(color='red'))

# Ajoute une couche des départements français à partir d'un fichier GeoJSON
fig.update_layout(mapbox_layers=[
    {
        "sourcetype": "geojson",
        # "source": "https://france-geojson.gregoiredavid.fr/repo/departements.geojson",
        "source": "https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb", 
        "type": "line",
        "color": "rgba(0, 0, 0, 0.9)",
        "line": {"width": 1}
    }
])
fig.update_layout(mapbox_style= "open-street-map", mapbox_zoom=5, mapbox_center = {"lat": 46.5, "lon": 2.}
                  )
# sauvegarde les fichiers
file_start= 'postes meteo-france '
# version CSV du fichier JSON 
df.to_excel(os.path.join(folder_out, file_start + ' - liste_' + now.strftime("%Y-%m-%d") + '.xlsx'), index=False)
# carte
file_map= file_start + '- carte_' + now.strftime("%Y-%m-%d")
fig.write_html(os.path.join(folder_out, file_map + '.html'))
fig.write_image(os.path.join(folder_out, file_map + '.png'))
fig.show()

In [62]:
# interroge les poste d'un département donné
departement= '83'

df[df['num_poste'].str.startswith(departement)]

,lat,lon,nom_usuel,num_poste,commune,ficheClimComplete,ficheClimReduite,alti
1951,43.794500,6.244833,AIGUINES_SAPC,83002004,AIGUINES,0.0,1.0,563
1953,43.641000,6.187500,AUPS,83007004,AUPS,0.0,1.0,497
1954,43.195167,6.379333,BORMES LES MIMOSAS,83019002,BORMES-LES-MIMOSAS,0.0,0.0,88
1975,43.200500,6.674500,CAP CAMARAT,83101001,RAMATUELLE,0.0,0.0,107
1985,43.079333,5.940833,CAP CEPET,83153001,SAINT-MANDRIER-SUR-MER,0.0,1.0,115
1958,43.261667,6.510333,COGOLIN_SAPC,83042001,COGOLIN,0.0,1.0,31
1959,43.232167,6.258000,COLLOBRIERES_SAPC,83043005,COLLOBRIERES,0.0,1.0,128
1960,43.658833,6.471333,COMPS-SUR-ARTUBY,83044003,MONTFERRAT,0.0,1.0,892
1961,43.247667,6.131333,CUERS,83049005,PIERREFEU-DU-VAR,0.0,1.0,72
1962,43.526167,6.453333,DRAGUIGNAN_SAPC,83050007,DRAGUIGNAN,0.0,1.0,173
